In [5]:
from geopy.geocoders import Nominatim
import plotly.graph_objects as go

import warnings
import itertools
import numpy as np
warnings.filterwarnings("ignore")
import pandas as pd

In [ ]:
df = pd.read_excel("C:\\Users\\ugogna\\Downloads\\Superstore.xls")
df.head()

In [6]:
def usAbbrivationCalc(df,State,Sales):
#dict to create US abbriviations
    try:    
        us_state_abbrev ={
                "Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA","Colorado": "CO"
                ,"Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA","Hawaii": "HI","Idaho": "ID"
                ,"Illinois": "IL","Indiana": "IN","Iowa": "IA","Kansas": "KS","Kentucky": "KY","Louisiana": "LA","Maine": "ME"
                ,"Maryland": "MD","Massachusetts": "MA","Michigan": "MI","Minnesota": "MN","Mississippi": "MS","Missouri": "MO"
                ,"Montana": "MT","Nebraska": "NE","Nevada": "NV","New Hampshire": "NH","New Jersey": "NJ","New Mexico": "NM"
                ,"New York": "NY","North Carolina": "NC","North Dakota": "ND","Ohio": "OH","Oklahoma": "OK","Oregon": "OR"
                ,"Pennsylvania": "PA","Rhode Island":"RI","South Carolina": "SC","South Dakota": "SD","Tennessee": "TN"
                ,"Texas": "TX","Utah": "UT","Vermont": "VT","Virginia": "VA","Washington": "WA","West Virginia": "WV","Wisconsin": "WI"
                ,"Wyoming": "WY","District of Columbia": "DC"
            }

        df_grouped =df.groupby([State])[Sales].sum().reset_index()
        text = df_grouped[State]
        df_grouped =df_grouped.replace({"State": us_state_abbrev}) 
    except Exception as error:
            raise Exception("usAbbrivationCalc:" + str(error))

    return (df_grouped,text)

In [7]:
def state_map_plot(df_grouped,text):
    data = [go.Choropleth(colorscale = 'YlGnBu',
                          autocolorscale = False,
                          locations = df_grouped['State'],
                          z = df_grouped['Sales'].astype(float),
                          locationmode = 'USA-states',
                          text= text,
                          marker = go.choropleth.Marker(
                                   line = go.choropleth.marker.Line(
                                          color = 'rgb(255,255,255)',
                                          width = 2
                                   )),
                          colorbar = go.choropleth.ColorBar(
                                     title = "Sales in $")
                          )]

    layout = go.Layout(
    title = go.layout.Title(
        text = 'Sales per state<br>(Hover for breakdown)'
    ),
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)'),
        #height=540 , width = 700, 
        paper_bgcolor="LightSteelBlue",
     )

    fig = go.Figure(data = data, layout = layout)
    #pyoff.plot(fig, filename= Config.SALES_FORECASTING_state_map_plot, auto_open=False)
    return(fig)

In [8]:
#function to calculate longitude latitude of location
def loc_lat_lan_calc(max_min_group,State_col):
    geolocator = Nominatim(user_agent="prediction")
    loc_lat=[]
    loc_lon=[]
    for i in max_group['State']:
        locate = geolocator.geocode(i)
        l1= locate.latitude 
        l2= locate.longitude
        loc_lat.append(l1)
        loc_lon.append(l1)
    retuen(loc_lat,loc_lon)

In [9]:
#map for lon_lat
def map_city_lon_lat(max_min_group,text,Sales_col):
    fig = go.Figure(data=go.Scattergeo(
        lon = loc_lon,
        lat = loc_lat,
        text = max_group['text'],
        mode = 'markers',
        marker_color = max_group['Sales'],
    ))
    fig.update_layout(
            title = 'Most Sales affected states US airports<br>(Hover for state names)'
                
    )
    fig.show()
    #pyoff.plot(fig, filename= Config.SALES_FORECASTING_state_map_plot, auto_open=False)

In [ ]:
con_city_group =pd.DataFrame({'Sales' :df.groupby(by=['Country','State'])['Sales'].sum()}).reset_index()
if ((con_city_group['Country'].nunique()) ==1):
    df_grouped,text =us_abbrivation(con_city_group,'State','Sales')
    fig2 =state_map_plot(df_grouped,text)
    fig2.show()
    
else :
    con_city_group['text'] = con_city_group['Country'] +  ', ' + con_city_group['State'] + '' + 'Sales: ' + con_city_group['Sales'].astype(str)
    con_city_group =con_city_group.sort_values(by='Sales', ascending=False).reset_index()
    max_group=con_city_group.nlargest(100,'Sales')
    min_group=con_city_group.nsmallest(100,'Sales')
    longtd, latidue = loc_lat_lan_calc(max_min_group,State_col)
    max_group['text'] = max_group['Country'] +  ', ' + max_group['State'] + '' + 'Sales: ' + max_group['Sales'].round(2).astype(str)
    map_city_lon_lat(max_min_group,text,Sales_col)
    

In [ ]:
df_grouped,text =us_abbrivation(df,'State','Sales')
